In [1]:
import pandas as pd 
titanic = pd.read_csv('../02_lesson/titanic/train.csv')

## Решение Lite

Разобьем пассажиров по группам:
- мололые (до 25 лет)
- взрослые (от 25 до 50 лет)
- старые (старше 50 лет)

Для этих целей напишем функцию, фильтрующую таблицу пассажиров по их возрастам:

In [2]:
def getAgePassanger(titanic, min_years = 0, max_years = 150):
    return titanic[(titanic.Age >= min_years) & (titanic.Age < max_years)]

In [3]:
total_passangers = titanic.PassengerId.count()
youngPassangers = titanic[titanic.Age < 25]
adultPassangers = titanic[(titanic.Age >= 25) & (titanic.Age < 50)]
oldPassangers = titanic[titanic.Age >= 50]

Напишем функцию, расчитывающую процент выживаемости для пассажиров титаника.
На вход функция принимает любую таблицу формата титаник

In [30]:
def getSurvPassByAge(titanicTable):
    p = titanicTable[titanicTable.Survived == 1].PassengerId.count() / titanicTable.PassengerId.count()
    return p

Расчитаем процент выживаемости для каждой группы пассажиров титаника, используя обобщенную функцию survivedTitanic и функцию извлечения пассажиров с нужным диапазоном возрастов. Процент выживаемости будет означать вероятность выживаемости пассажира в долях от единицы, в зависимости от группы его пользователя:

In [31]:
survivedYoung = getSurvPassByAge(getAgePassanger(titanic, max_years = 25))
survivedAdult = getSurvPassByAge(getAgePassanger(titanic, 25, 50))
oldPassangers = getSurvPassByAge(getAgePassanger(titanic, min_years=50))
ser = pd.Series({'young' : survivedYoung, 'adult' : survivedAdult})
summary = pd.DataFrame(['young', 'adult', 'old'])
summary['SurivedPer'] = pd.Series([survivedYoung, survivedAdult, oldPassangers])
summary

,0,SurivedPer
0,young,0.424460
1,adult,0.400552
2,old,0.364865


Вероятность выживания пассажира в зависимости от его возраста обратная - чем старше, тем вероятность ниже. Однако вероятность слабая.

## Решение Medium

**Задача:** Вычислить с какой вероятностью пассажир женского пола, если известно, что он выжил (использовать теорему Байеса)

Определим вероятности $B_1$ и $B_2$:

$P(B_1$) - Доля женщин на борту от общего числа пассажиров (вероятность что пассажир будет женщиной)

$P(B_2)$ - Доля мужчин на борту от общего числа пассажиров (вероятность что пассажир будет мужчиной)

In [66]:
passengersCount = titanic.PassengerId.count()
B1 = titanic[titanic.Sex == 'female'].PassengerId.count()/passengersCount
B2 = titanic[titanic.Sex == 'male'].PassengerId.count()/passengersCount
p = {'Вероятность пола' : pd.Series([B1, B2], index=["Female (B1)", "Male (B2)"])}
frame = pd.DataFrame(p)
frame

,Вероятность пола
Female (B1),0.352413
Male (B2),0.647587


Определим событие $A$ как выживаемость пассажира не зависимо от его пола (если пассажир выжил - происходит событие $A$)

Определим вероятности выживаемости пассажиров для каждого пола:

$P(A|B_1)$ - для женщин 

$P(A|B_2)$ - для мужчин 

In [64]:
titanicFemale = titanic[titanic.Sex == 'female']
titanicMale = titanic[titanic.Sex == 'male']
AB1 = titanicFemale[titanicFemale.Survived == 1].PassengerId.count() / titanicFemale.PassengerId.count()
AB2 = titanicMale[titanicMale.Survived == 1].PassengerId.count() / titanicMale.PassengerId.count()
p = {'Вероятность выживаемости' : pd.Series([AB1, AB2], index=["Female", "Male"])}
frame = pd.DataFrame(p)
frame

,Вероятность выживаемости
Female,0.742038
Male,0.188908


Запишем формулу полной вероятности для события A:
$$P(A) = P(B_1)P(A|B_1) + P(B_2)P(A|B_2)$$
Расчитаем вероятность события $A$ согласно полученной формуле:

In [67]:
PA = B1 * AB1 + B2 * AB2
PA

0.3838383838383838

In [28]:
femaleCount = titanic[titanic.Sex == 'female'].PassengerId.count()
probA = femaleCount / total_passangers
print('Вероятность события A:', round(probA, 2))

Вероятность события A: 0.35


Определим формулу байеса для события B если произошло событие A:
$$P(B_1|A) = \frac{P(B_1|A)P(B_1)}{P(A)}$$
Расчитаем вероятность наступления события $B_1$ исходя из формулы:

In [69]:
(AB1 * B1) / PA

0.6812865497076023

### **Ответ:** _Если пассажир выжил, вероятность того, что он женского пола составляет_ **68%**

### **Проверка** Выполним проверку получившейся вероятности. Расчитаем процент женщин, из числа выживших. Он должен совпадать с вероятностью выживания рассчитаной по формуле Байеса:

In [72]:
titanicSurvived = titanic[titanic.Survived == 1]
titanicSurvived[titanicSurvived.Sex == 'female'].PassengerId.count() / titanicSurvived.PassengerId.count()

0.6812865497076024

Процент выживших женщин совпадает в обоих моделях. Расчет верен.